In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.colors
import numpy as np
import os
import json
from ipywidgets import interact, widgets
import time
#plt.rcParams['text.usetex'] = True

import CuttingPlanes as CP

# Mixed Integer Linear Program (MILP)
$$
\begin{align}
&\min_{x} && c^T x\\
&\text{s.t.} && Ax \leq b\\
& && x \geq 0\\
& && x \in \mathbb{Z}^{n_1} \times \mathbb{R}^{n-n_1}
\end{align}
$$

In [2]:
# Specify Files
root_dir = "/Users/tobiaskohler/Uni/CuttingPlanes"
plots_dir = os.path.join(root_dir, 'plots')
data_dir = os.path.join(root_dir, 'data')
cpp_file = os.path.join(root_dir, 'build-CuttingPlanes-Desktop_arm_darwin_generic_mach_o_64bit-Debug/CuttingPlanes')
milp_file = os.path.join(data_dir, 'milp_wolsey_8-3.json')
res_file = os.path.join(data_dir, 'cp.json')

# Create a MILP
milp = CP.MILP(2, 1)
milp.set_objective([0, -1])
milp.add_inequality_constraint([3, 2], 6)
milp.add_inequality_constraint([-3, 1], 0)
#milp.add_inequality_constraint([-1, -1], -1)
milp.save('milp.json')

milp.load(milp_file)

# Run Cutting Plane Algorithm
b = os.popen(' '.join([cpp_file, milp_file, res_file])).read()
print(b)

# Add Nonnegativity Constraints for Visualization
milp.invert_constraints()
milp.add_inequality_constraint([1,0], 0, True)
milp.add_inequality_constraint([0,1], 0, True)
milp.check_dimensions()

# Load the Cutting Planes Solution(s) of the Simplex Algorithm
data = json.load(open(res_file))
sols = data['sols']
A_cuts = -np.array(data['cuts_coeffs'], dtype=np.float64)
b_cuts = -np.array(data['cuts_rhs'], dtype=np.float64)
n_cuts = len(b_cuts)

Cutting Planes
================ Mixed Integer Linear Program ================
n, m, p, n1: 2, 3, 0, 1
c:
  1 -10
A:
 -2 0.1
  0   1
  1   0
b:
0 2 3
B:

d:


Simplex: 0.1   2 with value -19.9
Add Cut -0.111111 0.0555556 <= 0
Simplex: 1 2 with value -19
Finished: 1 2 with value -19



In [3]:
# Save Plots
plotter = CP.Plotter()
imgs = []
for i in range(n_cuts+1):
    for j in range(2):
        title = milp.objective_str()
        img_file = os.path.join(plots_dir, '{i}{j}.png'.format(i=i,j=j))

        if i==0:
            sol = None if j==0 else sols[0]
            plotter.plot_milp(milp, title=title, optimal_solution=sol, save_file=img_file)
        else:
            sol = sols[i-1] if j==0 else sols[i]
            plotter.plot_milp(milp, title=title, optimal_solution=sol, cuts=(A_cuts[:i], b_cuts[:i]), save_file=img_file)

        img = mpimg.imread(img_file)
        imgs.append(img)

In [4]:
def show_plot(iter):
    plt.figure(figsize=(10,10))
    plt.axis('off')
    plt.imshow(imgs[iter])
    plt.show()

interact(show_plot, iter=widgets.IntSlider(min=0, max=2*n_cuts+1, step=1, value=0, description='Show Step'))

interactive(children=(IntSlider(value=0, description='Show Step', max=3), Output()), _dom_classes=('widget-int…

<function __main__.show_plot(iter)>